Test accuracy of convert_h2o.py by comparing its result with convert_h2o.m 

code_standard = stage-6

# py1.0 - STABLE - 13 June 2019 - tested by Eden Au

# 1. Setup

## 1.1 Initialization

In [1]:
import warnings
warnings.filterwarnings('ignore')

import os
import numpy as np
import xarray as xr
print('Packages imported.')

Packages imported.


## 1.2 Data Import

In [2]:
from utils_import_export import get_path_from_main, get_DS

# Get clean ARM data
path = get_path_from_main(suffix='data/clean/stage-6/arm_prec_sum_1_clean.cdf')
DS = get_DS(path)
DS

<xarray.Dataset>
Dimensions:        (p: 37, time: 3993, z: 512)
Coordinates:
  * time           (time) datetime64[ns] 1997-10-02T22:30:00 ... 2010-08-25T11:30:00
  * p              (p) float32 1000.0 975.0 950.0 925.0 ... 150.0 125.0 100.0
  * z              (z) float32 15.0 60.0 105.0 150.0 ... 22920.0 22965.0 23010.0
Data variables:
    prec_sfc_next  (time) float32 dask.array<shape=(3993,), chunksize=(3993,)>
    prec_sfc       (time) float32 dask.array<shape=(3993,), chunksize=(3993,)>
    T_sfc          (time) float32 dask.array<shape=(3993,), chunksize=(3993,)>
    rh_sfc         (time) float32 dask.array<shape=(3993,), chunksize=(3993,)>
    u_sfc          (time) float32 dask.array<shape=(3993,), chunksize=(3993,)>
    v_sfc          (time) float32 dask.array<shape=(3993,), chunksize=(3993,)>
    p_sfc          (time) float32 dask.array<shape=(3993,), chunksize=(3993,)>
    T_p            (time, p) float32 dask.array<shape=(3993, 37), chunksize=(3993, 37)>
    rh_p           (ti

In [3]:
# Pick the first instance
t = DS.sel(time = DS.time[0]).T_p.values
p = DS.sel(time = DS.time[0]).p.values
rh = DS.sel(time = DS.time[0]).rh_p.values

In [4]:
rh

array([71.63636 , 81.333336, 90.15    , 88.9     , 79.954544, 71.708336,
       71.4     , 43.258064, 59.058823, 65.55556 , 66.948715, 67.19512 ,
       67.414635, 64.82927 , 70.28261 , 77.333336, 79.75    , 69.520836,
       40.372547, 30.377777, 28.903847, 11.037037, 10.786885, 15.770492,
       21.171875, 23.050848, 30.6     , 35.815384, 46.472973, 53.635136,
       43.390804, 31.857143, 18.258427, 15.882353, 19.841667, 22.721428,
       23.94706 ], dtype=float32)

# 2. Testing

In [5]:
from convert_h2o import convert_h2o

## 2.1 Errors

In [6]:
# Wrong input flag okay
convert_h2o(p,t,rh,'wrong_input','N')

ValueError: Character flag for input unit is not defined.
Available choices are:
'M' - Mass Mixing Ratio (g/Kg),
'V' - Volume Mixing Ratio (ppv),
'H' - Relative Umidity (%),
'S' - Specific Humidity,
'D' - Dew Point (K),
'P' - Partial Pressure (hPa),
'N' - Number Density (cm-3), and
'R' - Mass Density (kg/m3).

In [7]:
# Wrong output flag okay
convert_h2o(p,t,rh,'N','wrong_output')

ValueError: Character flag for output unit is not defined.
Available choices are:
'M' - Mass Mixing Ratio (g/Kg),
'V' - Volume Mixing Ratio (ppv),
'H' - Relative Umidity (%),
'S' - Specific Humidity,
'D' - Dew Point (K),
'P' - Partial Pressure (hPa),
'N' - Number Density (cm-3),
'R' - Mass Density (kg/m3), and
'C' - Colummnar content (mm).

In [8]:
# Inconsistent input dim okay
convert_h2o(np.concatenate((p,p)),t,rh,'N','N')

ValueError: Input vectors have inconsistent dimensions: (74,), (37,), and (37,) respectively.

## 2.2 'H' - Relative Humidity & 'N' - Number Density

Checks were conducted by comparing the outputs between MATLAB and Python scripts 

We only have relative humidity ('H') data in hand, so let's convert it to number density ('N') first

In [9]:
# H2N okay
dennum = convert_h2o(p,t,rh,'H','N')
dennum

array([6.0478720e+17, 6.0584657e+17, 5.9297281e+17, 5.2649908e+17,
       4.3832667e+17, 3.7003562e+17, 3.4968778e+17, 2.1637391e+17,
       2.6827914e+17, 2.6081539e+17, 2.4363858e+17, 2.2311743e+17,
       2.0023924e+17, 1.7501701e+17, 1.6331428e+17, 1.5123107e+17,
       1.3447809e+17, 1.0359943e+17, 5.7276559e+16, 3.8790624e+16,
       3.1767554e+16, 1.0742791e+16, 9.0039453e+15, 1.0876181e+16,
       1.0824053e+16, 8.8774864e+15, 8.1153504e+15, 6.4539883e+15,
       5.3950174e+15, 3.8836498e+15, 1.8282386e+15, 8.3120193e+14,
       2.8248077e+14, 1.0528696e+14, 4.6968659e+13, 2.0624601e+13,
       1.4024043e+13], dtype=float32)

Test number density ('N') to itself

In [10]:
# N2N okay
convert_h2o(p,t,dennum,'N','N')

array([6.0478720e+17, 6.0584657e+17, 5.9297281e+17, 5.2649908e+17,
       4.3832667e+17, 3.7003562e+17, 3.4968778e+17, 2.1637391e+17,
       2.6827914e+17, 2.6081539e+17, 2.4363858e+17, 2.2311743e+17,
       2.0023924e+17, 1.7501701e+17, 1.6331428e+17, 1.5123107e+17,
       1.3447809e+17, 1.0359943e+17, 5.7276559e+16, 3.8790624e+16,
       3.1767554e+16, 1.0742791e+16, 9.0039453e+15, 1.0876181e+16,
       1.0824053e+16, 8.8774864e+15, 8.1153504e+15, 6.4539883e+15,
       5.3950174e+15, 3.8836498e+15, 1.8282386e+15, 8.3120193e+14,
       2.8248077e+14, 1.0528696e+14, 4.6968659e+13, 2.0624601e+13,
       1.4024043e+13], dtype=float32)

In [11]:
# N2H okay
np.abs(convert_h2o(p,t,dennum,'N','H') - rh).sum()

5.2452087e-05

## 2.3 'C' - Columnar Content (one-way)

Test number density ('N') to columnar content ('C'). This is one-way street as columnar content is a scalar and cannot convert to a vertical profile.

In [12]:
# N2C okay
convert_h2o(p,t,dennum,'N','C')

44.06498402343615

## 2.4 Other Variables

Convert ('N') to some unit, then convert it back to ('N'), and cross-check it with the MATLAB version

### 2.4.1 'M' Mass Mixing Ratio

In [14]:
# N2M okay
mmr = convert_h2o(p,t,dennum,'N','M')
mmr

array([1.5969711e+01, 1.6294048e+01, 1.6246408e+01, 1.4688289e+01,
       1.2468166e+01, 1.0759466e+01, 1.0429971e+01, 6.6171818e+00,
       8.4386845e+00, 8.4044523e+00, 8.0676184e+00, 7.5980215e+00,
       7.0122685e+00, 6.3155499e+00, 6.0674763e+00, 5.7864227e+00,
       5.3134723e+00, 4.2361946e+00, 2.4351444e+00, 1.7162538e+00,
       1.4637522e+00, 5.1701277e-01, 4.5378563e-01, 5.7493985e-01,
       5.9900719e-01, 5.1678991e-01, 4.9722856e-01, 4.1820747e-01,
       3.7120551e-01, 2.8545037e-01, 1.4438625e-01, 7.1487047e-02,
       2.6756154e-02, 1.1025394e-02, 5.5220397e-03, 2.8124100e-03,
       2.3547232e-03], dtype=float32)

In [22]:
# M2N okay
convert_h2o(p,t,mmr,'M','N')

array([6.0478720e+17, 6.0584657e+17, 5.9297281e+17, 5.2649908e+17,
       4.3832667e+17, 3.7003569e+17, 3.4968778e+17, 2.1637389e+17,
       2.6827912e+17, 2.6081538e+17, 2.4363860e+17, 2.2311742e+17,
       2.0023926e+17, 1.7501701e+17, 1.6331426e+17, 1.5123109e+17,
       1.3447809e+17, 1.0359942e+17, 5.7276559e+16, 3.8790624e+16,
       3.1767554e+16, 1.0742791e+16, 9.0039453e+15, 1.0876181e+16,
       1.0824053e+16, 8.8774869e+15, 8.1153504e+15, 6.4539883e+15,
       5.3950169e+15, 3.8836498e+15, 1.8282389e+15, 8.3120187e+14,
       2.8248077e+14, 1.0528696e+14, 4.6968659e+13, 2.0624599e+13,
       1.4024043e+13], dtype=float32)